# Quantum Simulator: Location Selector

This solution solves a problem where physical locations need to be equipped with services or products to satisfy demand within a certain area, while minimizing costs. For example, selecting branch and ATM sites in banking domain, selecting locations for CCTVs or Air Quality sensors in cities, etc. 
For large instances(number of locations) this problem cannot be optimally solved classically as it becomes computationally complex and time-consuming. Simulated Quantum Annealing provides a better alternative to classical methods by drastically reducing the computing time while minimizing cost. It delivers the required parallelization to explore many possible combinations simultaneously.

### Prerequisite

The kernel comes pre-installed with the required packages. Else ensure to have the following Python Packages in your environment at minimum:

    - numpy
    - pandas
### Usage instructions
Usage Methodology for the algorithm:
1. The input has to be a .csv file with the content in columns titled 'service', 'location', 'point', 'opening_costs', 'equip_costs'
1. The file should follow 'utf-8' encoding.
1. The input can have a maximum of 200 demand points, 200 locations and 5 services.
1. 'opening_costs' column should contain the opening costs, for each location, with 1st row having cost for 0th location, 2nd row having cost for 1st location and so on. 
1. 'equip_costs' column should contain costs of equipping a service, for each service, in same fashion as above.

General instructions for consuming the service on Sagemaker:
1. Access to AWS SageMaker and the model package
1. An S3 bucket to specify input/output
1. Role for AWS SageMaker to access input/output from S3


 ### Contents

1. [Importing libraries for runtime](#Importing-libraries-for-runtime)
1. [Model](#Model)
1. [Batch Transform](#Batch-Transform)
1. [Output](#Output)
1. [Endpoint](#Endpoint)

## Importing libraries for runtime

In [1]:
import pandas as pd
import boto3
import re

### Input Format
The input has to be a .csv file with the content in columns titled 

'service', 'location', 'point', 'opening_costs', 'equip_costs'.

Rows must be arranged in the manner as explained by following example. 

| service | location | point | opening_cost | equip_cost |
|---------|----------|-------|--------------|------------|
| 0       | 0        | 30    | 4807         | 457        |
| 0       | 0        | 48    | 4396         | 380        |
| 0       | 1        | 11    | 4029         |            |
| 0       | 1        | 14    | 4671         |            |
| 0       | 1        | 19    | 4323         |            |
...
| 1       | 2        | 13    |
| 1       | 2        | 17    |
...

Service 0 can be equipped at location 0 satifying demand points 30 and 48.
Service 0 can be equipped at location 1 satifying demand points 11, 14 and 19.
Service 1 can be equipped at location 2 satifying demand points 13 and 17.

Columns 'opening_costs' and 'equip_costs' are unrelated to first 3 columns. 
'opening_costs' contains opening costs for locations 0 to 49 (for a problem with 50 locations).
'equip_costs' contains equipping costs for services 0 and 1 (for a problem with 2 services).

<b> Note: 
    Input file should be of the form .csv and with 'utf-8' encoding. Ensure Content-Type is 'text/csv'
</b>

In [2]:
input_df  = pd.read_csv("input.csv")
input_df.head(10)

,service,location,point,opening_costs,equip_costs
0,0,0,30,4807.0,457.0
1,0,0,48,4396.0,380.0
2,0,1,11,4027.0,NaN
3,0,1,14,4671.0,NaN
4,0,1,19,4313.0,NaN
5,0,1,21,4758.0,NaN
6,0,1,37,4586.0,NaN
7,0,1,38,4066.0,NaN
8,0,1,44,4118.0,NaN
9,0,2,30,4734.0,NaN


## Model

### De-Serializing model

The serialzed Pickle file containing the trained model must be loaded for predicting the optimal path of the given set of customers and their demands.

The model is de-serialized to a Python object.


In [3]:
model_package_arn = 'Put your ARN here'

In [4]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [5]:
model = ModelPackage(model_package_arn=model_package_arn,
                    role = role,
                    sagemaker_session = sagemaker_session)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


## Batch Transform

In [6]:
import json 
import uuid


transformer = model.transformer(1, 'ml.m5.large')
transformer.transform('s3://mphasis-marketplace/location-selection/input.csv', content_type='text/csv')
transformer.wait()
#transformer.output_path
print("Batch Transform complete")
bucketFolder = transformer.output_path.rsplit('/')[3]

.......................2020-10-26T16:08:33.771:[sagemaker logs]: MaxConcurrentTransforms=1, MaxPayloadInMB=6, BatchStrategy=MULTI_RECORD
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [26/Oct/2020 16:08:33] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [26/Oct/2020 16:08:33] "#033GET /execution-parameters HTTP/1.1#033" 404 -
STEP 1 executed....
 * Serving Flask app "serve" (lazy loading)
 * Environment: production
   Use a production WSGI server instead.
 * Debug mode: off
 * Running on http://0.0.0.0:8080/ (Press CTRL+C to quit)
169.254.255.130 - - [26/Oct/2020 16:08:33] "#033GET /ping HTTP/1.1#033" 200 -
169.254.255.130 - - [26/Oct/2020 16:08:33] "#033GET /execution-parameters HTTP/1.1#033" 404 -
STEP 1 executed....

--------------------------------------------

BEST SOLUTION
 {'service_allocation': {0: [3, 4, 6, 8,

In [7]:
#print(s3bucket,s3prefix)
s3_conn = boto3.client("s3")
bucket_name="sagemaker-us-east-2-786796469737"
with open('FILE_NAME', 'wb') as f:
    s3_conn.download_fileobj(bucket_name, bucketFolder+'/input.csv.out', f)
    print("Output file loaded from bucket")

Output file loaded from bucket


## Output

- The processed output is of the form JSON file

- Output JSON file will contain allotted locations for each service, list of all enabled location and net cost of the solution.

In [8]:
import json
f = open("FILE_NAME",) 
data = json.load(f)
f.close() 

print('Service Allocations->')
for service, allocations in data['service_allocation'].items():
    print('Service:', service, '\tLocations: ', allocations)

print('\nEnabled Locations-> ', data['enabled_locations'])
print('\nNet Cost-> ', data['cost'])

Service Allocations->
Service: 0 	Locations:  [3, 4, 6, 8, 10, 11, 12, 15, 17, 19, 20, 23, 24, 27, 28, 30, 31, 32, 34, 35, 36, 37, 39, 40, 42, 43, 44, 45, 47]
Service: 1 	Locations:  [0, 1, 2, 6, 9, 10, 11, 15, 16, 19, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 38, 40, 41, 43, 44, 45, 47, 48]

Enabled Locations->  [0, 1, 2, 3, 4, 6, 8, 9, 10, 11, 12, 15, 16, 17, 19, 20, 22, 23, 24, 26, 27, 28, 29, 30, 31, 32, 34, 35, 36, 37, 38, 39, 40, 41, 42, 43, 44, 45, 47, 48]

Net Cost->  235963.0


## Endpoint
Here is a sample endpoint for reference

In [9]:
import json 
import uuid
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role
from sagemaker import ModelPackage
import boto3
from IPython.display import Image
from PIL import Image as ImageEdit

role = get_execution_role()

sagemaker_session = sage.Session()
bucket=sagemaker_session.default_bucket()

In [10]:
content_type='text/csv'
model_name='LOCATIONSET'
real_time_inference_instance_type='ml.m5.xlarge'

In [11]:
model_package_arn = 'Put your ARN here'

In [12]:
from sagemaker import ModelPackage
import sagemaker as sage
from sagemaker import get_execution_role

role = get_execution_role()
sagemaker_session = sage.Session()

In [13]:
#Define predictor wrapper class
def predict_wrapper(endpoint, session):
    return sage.RealTimePredictor(endpoint, session,content_type=content_type)
#create a deployable model from the model package.
model = ModelPackage(role=role,
                    model_package_arn=model_package_arn,
                    sagemaker_session=sagemaker_session,
                    predictor_cls=predict_wrapper)

Parameter image will be renamed to image_uri in SageMaker Python SDK v2.


In [14]:
predictor = model.deploy(1, real_time_inference_instance_type, endpoint_name=model_name)

-----------!

In [15]:
input_file_name = 'input.csv'
output_file_name = 'output.json'

In [16]:
!aws sagemaker-runtime invoke-endpoint --endpoint-name $model_name --body fileb://$input_file_name --content-type 'text/csv' --region us-east-2 $output_file_name

{
    "ContentType": "application/json",
    "InvokedProductionVariant": "AllTraffic"
}


In [17]:
f = open(output_file_name,) 
data = json.load(f)
f.close() 

print('Service Allocations->')
for service, allocations in data['service_allocation'].items():
    print('Service:', service, '\tLocations: ', allocations)

print('\nEnabled Locations-> ', data['enabled_locations'])
print('\nNet Cost-> ', data['cost'])

Service Allocations->
Service: 0 	Locations:  [1, 2, 3, 4, 5, 10, 11, 16, 17, 23, 24, 27, 32, 36, 37, 38, 40, 41, 42, 44, 46, 47, 48]
Service: 1 	Locations:  [0, 3, 7, 8, 9, 10, 11, 13, 15, 19, 21, 23, 24, 30, 31, 34, 36, 38, 40, 44, 46, 47]

Enabled Locations->  [0, 1, 2, 3, 4, 5, 7, 8, 9, 10, 11, 13, 15, 16, 17, 19, 21, 23, 24, 27, 30, 31, 32, 34, 36, 37, 38, 40, 41, 42, 44, 46, 47, 48]

Net Cost->  225388.0


In [18]:
# Delete Endpoint
predictor.delete_endpoint()